In [46]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
from surprise.model_selection import GridSearchCV
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pickle


In [47]:
ratings = pd.read_csv('../data/ratings.csv')
movies = pd.read_csv('movies_feature_engineered.csv')
ratings['rating'] = ratings['rating']*2

In [48]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [49]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movieId          9742 non-null   int64  
 1   popularity       9742 non-null   float64
 2   runtime          9742 non-null   float64
 3   vote_average     9742 non-null   float64
 4   vote_count       9742 non-null   float64
 5   Action           9742 non-null   int64  
 6   Adventure        9742 non-null   int64  
 7   Animation        9742 non-null   int64  
 8   Children         9742 non-null   int64  
 9   Comedy           9742 non-null   int64  
 10  Crime            9742 non-null   int64  
 11  Documentary      9742 non-null   int64  
 12  Drama            9742 non-null   int64  
 13  Family           9742 non-null   int64  
 14  Fantasy          9742 non-null   int64  
 15  History          9742 non-null   int64  
 16  Horror           9742 non-null   int64  
 17  IMAX          

In [50]:
merged_df = ratings.merge(movies, on='movieId', how='left')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 31 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   userId           100836 non-null  int64  
 1   movieId          100836 non-null  int64  
 2   rating           100836 non-null  float64
 3   timestamp        100836 non-null  int64  
 4   popularity       100836 non-null  float64
 5   runtime          100836 non-null  float64
 6   vote_average     100836 non-null  float64
 7   vote_count       100836 non-null  float64
 8   Action           100836 non-null  int64  
 9   Adventure        100836 non-null  int64  
 10  Animation        100836 non-null  int64  
 11  Children         100836 non-null  int64  
 12  Comedy           100836 non-null  int64  
 13  Crime            100836 non-null  int64  
 14  Documentary      100836 non-null  int64  
 15  Drama            100836 non-null  int64  
 16  Family           100836 non-null  int6

In [51]:
user_enc = LabelEncoder()
movie_enc = LabelEncoder()

merged_df['user_idx'] = user_enc.fit_transform(ratings['userId'])
merged_df['movie_idx'] = movie_enc.fit_transform(merged_df['movieId'])

num_users = merged_df['user_idx'].nunique()
num_movies = merged_df['movie_idx'].nunique()

In [52]:
1076 in list(movies['movieId'].unique())

True

In [53]:
pickle.dump(user_enc, open("user_enc.pkl", "wb"))
pickle.dump(movie_enc, open("movie_enc.pkl", "wb"))

In [54]:
num_features = ['vote_average', 'vote_count', 'popularity', 'runtime']
scaler = StandardScaler()
merged_df[num_features] = scaler.fit_transform(merged_df[num_features])

In [55]:
merged_df.head()

,userId,movieId,rating,timestamp,popularity,runtime,vote_average,vote_count,Action,Adventure,...,Mystery,Romance,Sci-Fi,Science Fiction,TV Movie,Thriller,War,Western,user_idx,movie_idx
0,1,1,8.0,964982703,2.030276,-1.396378,1.162476,1.913321,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,8.0,964981247,-0.757716,-0.563646,-0.655004,-0.809518,0,0,...,0,1,0,0,0,0,0,0,0,2
2,1,6,8.0,964982224,1.170748,2.309283,1.101852,0.257766,1,0,...,0,0,0,0,0,0,0,0,0,5
3,1,47,10.0,964983815,1.913863,0.518907,1.666025,2.339322,0,0,...,1,0,0,0,0,1,0,0,0,43
4,1,50,10.0,964982931,0.291648,-0.355462,1.414869,0.710271,0,0,...,0,0,0,0,0,1,0,0,0,46


In [56]:
cat_features = ['user_idx', 'movie_idx']
cont_features = ['popularity', 'runtime',
       'vote_average', 'vote_count', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'History', 'Horror', 'IMAX', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Science Fiction', 'TV Movie', 'Thriller', 'War',
       'Western'] 
X_cat = merged_df[cat_features].values  # (N, 2)
X_cont = merged_df[cont_features].values.astype('float32')  # (N, C)
y = merged_df['rating'].values.astype('float32')  # (N,)

In [57]:
class RatingDataset(Dataset):
    def __init__(self, X_cat, X_cont, y):
        self.X_cat = torch.tensor(X_cat, dtype=torch.long)
        self.X_cont = torch.tensor(X_cont, dtype=torch.float)
        self.y = torch.tensor(y, dtype=torch.float)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X_cat[idx], self.X_cont[idx], self.y[idx]

dataset = RatingDataset(X_cat, X_cont, y)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)


In [58]:
class FactorizationMachineModel(nn.Module):
    def __init__(self, num_users, num_movies, num_cont_features, emb_dim=32):
        super().__init__()

        # Embedding layers
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.movie_emb = nn.Embedding(num_movies, emb_dim)

        # Linear (first-order) terms for categorical
        self.user_bias = nn.Embedding(num_users, 1)
        self.movie_bias = nn.Embedding(num_movies, 1)

        # Linear for continuous
        self.linear_cont = nn.Linear(num_cont_features, 1)

        self.dropout = nn.Dropout(0.2)
        self.output = nn.Linear(1, 1)  # optional non-linearity head

    def forward(self, X_cat, X_cont):
        user_idx, movie_idx = X_cat[:, 0], X_cat[:, 1]

        user_vec = self.user_emb(user_idx)
        movie_vec = self.movie_emb(movie_idx)

        # ----- FM interaction (dot product of embeddings) -----
        interaction = torch.sum(user_vec * movie_vec, dim=1, keepdim=True)

        # ----- Linear terms -----
        linear_cat = self.user_bias(user_idx) + self.movie_bias(movie_idx)
        linear_cont = self.linear_cont(X_cont)

        # ----- Final output -----
        out = interaction + linear_cat + linear_cont
        out = self.output(self.dropout(out))

        return out.squeeze(1)


In [59]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))


In [60]:
model = FactorizationMachineModel(
    num_users=num_users,
    num_movies=num_movies,
    num_cont_features=X_cont.shape[1],
    emb_dim=32
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [61]:
def train_model(model, dataloader, epochs=20):
    model.train()
    device = next(model.parameters()).device

    for epoch in range(epochs):
        total_loss = 0.0

        for X_cat, X_cont, y in tqdm(dataloader):
            X_cat = X_cat.to(device)
            X_cont = X_cont.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            preds = model(X_cat, X_cont)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * len(y)

        avg_loss = total_loss / len(dataloader.dataset)
        print(f"Epoch {epoch+1}/{epochs} — Loss: {avg_loss:.4f}")


In [62]:
train_model(model, dataloader)


100%|██████████| 197/197 [00:04<00:00, 43.43it/s] 


Epoch 1/20 — Loss: 71.6706


100%|██████████| 197/197 [00:01<00:00, 124.09it/s]


Epoch 2/20 — Loss: 51.1824


100%|██████████| 197/197 [00:01<00:00, 109.74it/s]


...

Epoch 18/20 — Loss: 8.5426


100%|██████████| 197/197 [00:01<00:00, 120.61it/s]


Epoch 19/20 — Loss: 8.0796


100%|██████████| 197/197 [00:01<00:00, 110.50it/s]

Epoch 20/20 — Loss: 7.6134


In [63]:
from sklearn.model_selection import train_test_split

X_cat_train, X_cat_val, X_cont_train, X_cont_val, y_train, y_val = train_test_split(
    X_cat, X_cont, y, test_size=0.2, random_state=42)


In [64]:
train_dataset = RatingDataset(X_cat_train, X_cont_train, y_train)
val_dataset = RatingDataset(X_cat_val, X_cont_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)


In [65]:

def evaluate_model(model, val_loader):
    model.eval()
    device = next(model.parameters()).device

    all_preds = []
    all_targets = []
    def accuracy_within_threshold(y_true, y_pred, threshold=1.0):
        return np.mean(np.abs(y_true - y_pred) <= threshold)

    with torch.no_grad():
        for X_cat, X_cont, y in val_loader:
            X_cat = X_cat.to(device)
            X_cont = X_cont.to(device)
            y = y.to(device)

            preds = model(X_cat, X_cont)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(y.cpu().numpy())

    rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
    mae = mean_absolute_error(all_targets, all_preds)
    acc_05 = accuracy_within_threshold(np.array(all_targets), np.array(all_preds), threshold=0.5)
    acc_1 = accuracy_within_threshold(np.array(all_targets), np.array(all_preds), threshold=1.0)
    acc_2 = accuracy_within_threshold(np.array(all_targets), np.array(all_preds), threshold=2.0)

    print(f"Accuracy within 0.5: {acc_05:.4f}")
    print(f"Accuracy within 1.0: {acc_1:.4f}")
    print(f"Accuracy within 2.0: {acc_2:.4f}")
    print(f"Validation RMSE: {rmse:.4f}")
    print(f"Validation MAE : {mae:.4f}")


In [66]:
model = FactorizationMachineModel(
    num_users=num_users,
    num_movies=num_movies,
    num_cont_features=X_cont.shape[1],
    emb_dim=64
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [67]:
train_model(model, train_loader, epochs=50)


100%|██████████| 158/158 [00:01<00:00, 93.87it/s] 


Epoch 1/50 — Loss: 10.7132


100%|██████████| 158/158 [00:01<00:00, 112.68it/s]


Epoch 2/50 — Loss: 9.0871


100%|██████████| 158/158 [00:01<00:00, 111.91it/s]


...

Epoch 48/50 — Loss: 1.5990


100%|██████████| 158/158 [00:01<00:00, 99.83it/s] 


Epoch 49/50 — Loss: 1.5631


100%|██████████| 158/158 [00:01<00:00, 107.10it/s]

Epoch 50/50 — Loss: 1.5361


In [68]:
evaluate_model(model, val_loader)

Accuracy within 0.5: 0.1931
Accuracy within 1.0: 0.3758
Accuracy within 2.0: 0.6708
Validation RMSE: 2.1091
Validation MAE : 1.6617


In [69]:
model = FactorizationMachineModel(
    num_users=num_users,
    num_movies=num_movies,
    num_cont_features=X_cont.shape[1],
    emb_dim=64
).to("cuda" if torch.cuda.is_available() else "cpu")

criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [70]:
train_model(model, dataloader, epochs=50)

100%|██████████| 197/197 [00:01<00:00, 104.62it/s]


Epoch 1/50 — Loss: 9.6338


100%|██████████| 197/197 [00:02<00:00, 97.31it/s] 


Epoch 2/50 — Loss: 8.0065


100%|██████████| 197/197 [00:01<00:00, 115.37it/s]


...

Epoch 48/50 — Loss: 1.4360


100%|██████████| 197/197 [00:01<00:00, 99.91it/s] 


Epoch 49/50 — Loss: 1.4095


100%|██████████| 197/197 [00:01<00:00, 103.73it/s]

Epoch 50/50 — Loss: 1.3923


In [71]:
torch.save(model.state_dict(), "fm_model.pt")
pickle.dump(user_enc, open("user_enc.pkl", "wb"))
pickle.dump(movie_enc, open("movie_enc.pkl", "wb"))

# Also export the cont_features list
import json
with open("cont_features.json", "w") as f:
    json.dump(cont_features, f)
